<h1> Probabilistic Reasoning </h1>

$ \textbf{Exercise 1: Follow the robot} $

Assume you get your hands on a robot that has various sensors.

$\textbf{a)}$ The robot has a very cheap camera on board, so it's not very accurate at reading colors. After color calibration you know the camera color model. Assume the robot is located in a white room with 5 boxes: 2 red, 2 green and a blue one. The robot moves towards a box and the camera reads green. How likely is it that the box is actually green? 

Let us first write down the color mapping as a 2d numpy array.

In [6]:
import numpy as np

color_model = np.array([[0.8, 0.1, 0.1],
                         [0.1, 0.6, 0.2],
                         [0.1, 0.3, 0.7]])

print(color_model)

[[ 0.8  0.1  0.1]
 [ 0.1  0.6  0.2]
 [ 0.1  0.3  0.7]]


Let us also for convenience define some index variables to access the array entries. 

In [7]:
R, G, B = 0, 1, 2

The quantities of the boxes are:

In [8]:
quantities = np.array([2, 2, 1])
print(quantities)

[2 2 1]


To compute the asked probability, we will use Bayes' rule:

$$p(x=G|z=G) = p(z=G|x=G) \frac{p(x=G)}{p(z=G)}$$

Now let's compute each term. 
The first term $p(z=G|x=G)$ is the <b>likelihood</b> and is exactly why we need a sensor model.

In [9]:
p_readGreenWhileGreen = color_model[G][G]
print(p_readGreenWhileGreen)

0.6


The second term is the <b>prior</b>. In this case we know the prior exactly: $$p(x=G) = \frac{n_G}{n_R + n_G + n_B} = \frac{2}{5}$$

In [10]:
p_isGreen = quantities[G] / sum(quantities)
print(p_isGreen)

0.4


The denominator is the <b>normalizer</b> and we compute it by marginalization: $$p(z=G) = p(z=G|x=R)p(x=R) + p(z=G|x=G)p(x=G) + p(z=G|x=B)p(x=B) = \sum_{X \in \{R,G,B\}} p(z=G|x=X)p(x=X)$$

The last sum is just an inner product. The first vector is given by the row in the color mapping that corresponds to observing the color green. It is the likelihood of each color when reading green. The second vector is just the prior probability of each color:

In [12]:
p_likelihoods = color_model[G]

In [13]:
p_isColor = quantities / sum(quantities)
print(p_isColor)

[ 0.4  0.4  0.2]


Therefore, the probability of sensing green is:

In [14]:
p_readGreen = np.dot( p_likelihoods, p_isColor)
print(p_readGreen)

0.32


Now we have everything we need. The probability that the box is actually green is:

In [15]:
p_isGreenWhileReadGreen = p_readGreenWhileGreen * p_isGreen / p_readGreen
print(p_isGreenWhileReadGreen)

0.75


$\textbf{b)}$ The robot has a proximity sensor and it uses it to measure its distance from a door. The sensor can be modeled using a continuous random variable
with a Normal distribution with $\sigma_1 = 0.3$. Express the sensor model $p(z_t|x_t)$ in the
full form (not the shorthand notation).

$$ p(z_t|x_t) = \cal{N} (z_t ; x_t, \sigma_1^2)
= \frac{1}{\sigma_1\sqrt{2\pi}} \exp \left( - \frac{(z_t-x_t)^2}{2\sigma_1^2} \right) 
= \frac{1}{0.3\sqrt{2\pi}} \exp \left( - \frac{(z_t-x_t)^2}{2 \cdot 0.3^2} \right) $$

$\textbf{c)}$ Now the robot moves into a hallway. Initially it knows it is located
at the door ($x_0=0$). The robot can execute move commands but the result of the
action is not always perfect. Assume that the robot moves with constant speed $v$.
The motion can also be modeled with a Gaussian with deviation $\sigma_2 = 0.1$. Write
the motion model $p(x_t|x_{t−1}, u_t)$.

The motion can also be modeled with a Gaussian. We just need to think about what
is our mean and what is our variance. The variance is the square of the standard deviation which given as the actuator noise
$\sigma_2 = 0.1$. Our mean is the position we expect the robot to be at, after the motion
$u_t$. Since our robot moves with constant speed $v$, the expected position is simply
$\mu = x_{t−1} + v \Delta t $. Therefore we have

$$ p(x_t|x_{t-1}, u_t) = \cal{N} (x_t; x_{t-1} + v \Delta t, \sigma_2^2)
= \frac{1}{\sigma_2 \sqrt{2\pi}} \exp \left( - \frac{(x_t - (x_{t-1}+ v \Delta t))^2}{2\sigma_2^2} \right) 
= \frac{1}{0.1\sqrt{2\pi}} \exp \left( - \frac{(x_t - (x_{t-1}+ v \Delta t))^2}{2 \cdot 0.1^2} \right) $$

$\textbf{d)}$ You let the robot run with a speed of 1 m/s. The robot only runs
forward and it updates its belief every second. Assume you get the following sensor
measurements in the first 3 seconds: $\{z_1 = 1.2, z_2 = 1.6, z_3 = 2.5\}$.
Further assume that the hallway is only 5 meters long. Where
does the robot believe it is located with respect to the door after 3 seconds? How
certain is it about its location?

We model the state variable $x$ as a continuous random variable with values between 0
and 5, where 0 means that the robot is at the door. We want to compute the robot’s
belief. Initially, the robot knows it is located at the door ($x_0=0$), therefore we have
$Bel(x_0 = 0) = +\infty $. We then use the Bayes Filter algorithm to compute the belief after
3 seconds, namely $Bel(x_3)$. Since this is a recursive algorithm we have to compute the
belief at each time step. The general equation of the Bayes filter is:

$$ Bel(x_t) = \eta_t \cdot p(z_t|x_t) \cdot \int p(x_t | x_{t-1}, u_t) \cdot Bel(x_{t-1}) dx_{t-1} $$

In our case the space can be reduced to an interval from 0 to 5, so we get a definite integral. Furthermore, there is only one action $u$ that is taken at every time step, which is to move with constant speed 1 m/s.
Let us see how we would compute the belief after one second: |

$$ Bel(x_1) = \eta_1 \cdot p(z_1|x_1) \cdot \int_0^5 p(x_1 | x_{0}, u) \cdot Bel(x_{0}) dx_0 $$ 

which reduces to $$ Bel(x_1) = \eta_1 \cdot p(z_1|x_1) \cdot p(x_1 | x_{0}=0, u)  $$

We have the sensor model from question 1b and the motion model from question 1c, so we can substitute with the normal distributions:

$$ p(z_1|x_1) = \frac{1}{0.3\sqrt{2\pi}} \exp \left( - \frac{(1.2-x_1)^2}{2 \cdot 0.3^2} \right) $$
              
$$ p(x_1 | x_0, u) = \frac{1}{0.1\sqrt{2\pi}} \exp \left( - \frac{(x_1 - (0 + 1 \cdot 1))^2}{2 \cdot 0.1^2} \right) $$

$$ \eta_1 = \left( \int_0^5 p(z_1=1.2|x_1) \cdot p(x_1|x_0=0, u) dx_1 \right)^{-1} $$

In our example, the sensor model and the action model are both represented by Gaussian pdfs (in 1-D). This is a particular instance of Bayes Filter algorithms, namely the Kalman Filter. The great advantage of the Kalman Filter is that we can do all computations in closed form:

The integral $\int p(x_t | x_{t-1}, u_t) \cdot Bel(x_{t-1}) dx_{t-1}$ is a convolution of Gaussian pdfs. It turns out that this results to another Gaussian pdf with extremely easy to compute parameters:

$$ \int \cal N(x ; \mu_1, \sigma_1^2) \cdot \cal N(x ; \mu_2, \sigma_2^2) dx = \cal N(x ; \mu_1 + \mu_2, \sigma_1^2 + \sigma_2^2) $$

From this follows that the product of the sensor model with the integral is a product of two Gaussian pdfs which also results in a Gaussian pdf:

$$ \cal N(x ; \mu_1, \sigma_1^2) \cdot \cal N(x ; \mu_2, \sigma_2^2) = \cal N \left( x ; \frac{\sigma_2^2 \mu_1 + \sigma_1^2 \mu_2}{\sigma_1^2 + \sigma_2^2}, \frac{\sigma_1^2 \sigma_2^2}{\sigma_1^2 + \sigma_2^2} \right) $$

Therefore, every action and measurement update is extremely efficient as we only have to compute the new parameters of the Gaussians. If the models were arbitrary distributions, we would have to compute the updates explicitly.<br> See also the full code examples for both cases written in python.

$ \textbf{Exercise 2: An overview of ML methods} $

Try to find (for example by internet search or from the book (Bishop)) at least
5 examples for learning techniques that have not been discussed in class. Describe these
techniques briefly and classify them with respect to the categories presented in the lecture.
Here are some examples of learning algorithms:

<ol>
<li> Mean-shift clustering: Unsupervised learning </li>
<li> Perceptron algorithm: Discriminant function</li>
<li> Neural Networks: Discriminative model</li>
<li> Bayes classifier: Generative model</li>
<li> Conditional Random Field: Discriminative model</li>
<li> AdaBoost: Discriminant function</li>
</ol>

For a detailed explanation, please see the textbook Pattern Recognition and Machine
Learning by C.M. Bishop or the slides.